In [1]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [2]:
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_1720/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [16]:
# 显示所有的列
all_columns = list(all_data.columns)
all_data[all_columns[79:]]

,version,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1
0,1,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220821
1,1,7.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":...","['{""companyName"": ""北京美餐好客科技有限公司"", ""companyNatu...","['{""description"": ""项目描述: 母婴类人群，包含童装、童鞋、奶粉。\\n运...",20220821
2,1,14.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":...","['{""companyName"": ""Continental Tires (China) L...",NaN,20220821
3,1,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220821
4,1,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220821
...,...,...,...,...,...,...,...
81291,1,1.0,NaN,"['{""endDate"": ""2022-07"", ""unified"": false, ""de...","['{""endDate"": ""2022-07"", ""companyName"": ""江门市新会...","['{""name"": null}']",20220821
81292,1,23.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""1999-07"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""上海微创医...","['{""name"": null}']",20220821
81293,1,23.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""1999-07"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""上海微创医...","['{""name"": null}']",20220821
81294,1,5.0,NaN,"['{""endDate"": ""2015-06"", ""unified"": false, ""de...","['{""endDate"": ""2020-07"", ""companyName"": ""金山云"",...","['{""name"": null}']",20220821


In [4]:
# 注意一下，这里的列名和sample pipeline中的列名有一点大小写区别，问题不大
cv_columns = ['cv_id', 'currentPosition', 'desiredPosition', 'industry', 'desiredIndustry', 'majorName', 'skills', 'eduTracks', 'jobTracks', 'projectTracks']
jd_columns = ['jd_id', 'title', 'category_name', 'tags', 'description', 'requirement']

对于上述文本字段的一些看法:
* industry和desiredIndustry的缺失值比较严重
* majorName感觉字段错位，里面有很多学校信息.建议重新入库。 

In [17]:
jd_data = all_data[jd_columns]
jd_data

,jd_id,title,category_name,tags,description,requirement
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...
...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...


In [25]:
cv_data = all_data[cv_columns]
cv_data

,cv_id,currentPosition,desiredPosition,industry,desiredIndustry,majorName,skills,eduTracks,jobTracks,projectTracks
0,2c9207157bf91042017c0d2beef00470,副总裁/副总经理,NaN,NaN,NaN,国际经济与贸易,"[""Ipd"",""App"",""Kpi"",""Crm"",""View"",""优化"",""策划"",""营销""...","['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc..."
1,2c9207157bf91042017c0d979cbb047d,数字化营销经理,NaN,餐饮业,NaN,电气工程及其自动化,"[""H5"",""C1"",""Seo"",""Sem"",""Top"",""Kpi"",""Crm"",""优化"",...","['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":...","['{""companyName"": ""北京美餐好客科技有限公司"", ""companyNatu...","['{""description"": ""项目描述: 母婴类人群，包含童装、童鞋、奶粉。\\n运..."
2,2c9207157bf91042017c7232fbab3be1,NaN,NaN,NaN,NaN,Marketing,"[""Based"",""Control"",""Sap Crm"",""Improve"",""Indivi...","['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":...","['{""companyName"": ""Continental Tires (China) L...",NaN
3,2c9207157d308592017d4a94cb1113a1,泰国国家经理,NaN,NaN,NaN,企业管理,"[""App"",""Resume"",""运营"",""策划"",""营销"",""搭建"",""笔译"",""舞蹈"",...","['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主..."
4,2c9207157dabb0a3017dbca16cdf4c26,项目经理,项目经理/主管,互联网+,"智能硬件,互联网+",英语,"[""智能硬件""]","['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20..."
...,...,...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,生产工、包装工,生产制造/机械设计/制造/机械设备工程师,NaN,NaN,无,电气,"['{""endDate"": ""2022-07"", ""unified"": false, ""de...","['{""endDate"": ""2022-07"", ""companyName"": ""江门市新会...","['{""name"": null}']"
81292,8a69d7c282199453018247bac4a36859,高级市场总监,市场/市场/营销/市场营销,NaN,NaN,上海大学,"搭建,翻译,功能,医学,数据库,Education,产品管理,产品经理,医疗器械,英语听说读写","['{""schoolTag"": ""211"", ""endDate"": ""1999-07"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""上海微创医...","['{""name"": null}']"
81293,8a69d7c282199453018247bac4a36859,高级市场总监,市场/市场/营销/市场营销,NaN,NaN,上海大学,"搭建,翻译,功能,医学,数据库,Education,产品管理,产品经理,医疗器械,英语听说读写","['{""schoolTag"": ""211"", ""endDate"": ""1999-07"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""上海微创医...","['{""name"": null}']"
81294,8a69d7db826df6c201827b4338cd60c2,java,技术/后端开发/Java,NaN,NaN,太原理工大学,"Bi,Cdn,Elk,Git,Yii,App,Php,Ext,Etl,Java,Sina,H...","['{""endDate"": ""2015-06"", ""unified"": false, ""de...","['{""endDate"": ""2020-07"", ""companyName"": ""金山云"",...","['{""name"": null}']"


In [19]:
cv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81296 entries, 0 to 81295
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cv_id            81296 non-null  object
 1   currentPosition  78952 non-null  object
 2   desiredPosition  51408 non-null  object
 3   industry         15671 non-null  object
 4   desiredIndustry  9345 non-null   object
 5   majorName        80395 non-null  object
 6   skills           74527 non-null  object
 7   eduTracks        81293 non-null  object
 8   jobTracks        80324 non-null  object
 9   projectTracks    55283 non-null  object
dtypes: object(10)
memory usage: 6.2+ MB


In [23]:
with open('./majorname.txt', 'w') as fo:
    for major in cv_data['majorName']:
        fo.write("{}\n".format(major))


jd可以做3个向量
* title + category_name + tags
* description
* requirement

cv可以做4个向量：
* currentPosition + desiredPosition
* skills
* jobTracks
* projectTracks

向量1：title + category_name + tags

In [ ]:
# step1：缺失值填充

jd_data['title'].fillna('', inplace=True)
jd_data['category_name'].fillna('', inplace=True)
jd_data['tags'].fillna('[]', inplace=True)

In [ ]:
# step2：定义新列，将title + category_name合并

col_name1 = 'title'
col_name2 = 'category_name'

def col_merge_str_fun(series):
    '''
    适用于文本列合并
    '''
    return series[col_name1] + ' ' + series[col_name2]

jd_data['title_category'] = jd_data.apply(col_merge_str_fun, axis=1)
jd_data

In [ ]:
# step3：将新列title_category通过jieba分词处理成列表

col_name = 'title_category'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['title_category_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


In [ ]:
# step4：将tags列通过jieba分词处理成列表

col_name = 'tags'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['tags_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


In [ ]:
# step5：将title_category_jieba和tags_jieba两个列表合并

col_name1 = 'title_category_jieba'
col_name2 = 'tags_jieba'

def col_merge_list_fun(series):
    '''
    适用于列表列合并
    '''
    return series[col_name1] + series[col_name2]

jd_data['title_category_tags_jieba'] = jd_data.apply(col_merge_list_fun, axis=1)
jd_data


In [ ]:
# step6：过滤title_category_tags_jieba, 得到以空格分割的字符串

col_name = 'title_category_tags_jieba'
def col_jieba_filter_fun(series):

    col_list_filter = []
    # 得到tag列表
    col_list = series[col_name]
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

jd_data['title_category_tags_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)
jd_data

In [ ]:
# step7：将title_category_tags_jieba_filter转成tfidf向量

def get_tfidf(df, col_name):
    # col_name必须是以空格分割的字符串
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    # 返回的是nparray
    vector = vectorizer.fit_transform(text)
    return pd.DataFrame(vector.toarray()), vectorizer

tfidf, vectorizer = get_tfidf(jd_data, 'title_category_tags_jieba_filter')
tfidf


In [ ]:
vectorizer.vocabulary_

In [ ]:
# step8：将tfidf降维到n维

def get_tfidf_pca(tfidf, n=20):
    pca = PCA(n_components=n)
    tfidf_pca = pca.fit_transform(tfidf)
    tfidf_pca = pd.DataFrame(tfidf_pca)
    return tfidf_pca

tfidf_pca = get_tfidf_pca(tfidf, 10)
tfidf_pca

向量2：description

In [ ]:
# step1：缺失值填充

jd_data['description'].fillna('', inplace=True)

In [ ]:
# step2：将desciption列通过jieba分词处理成列表

col_name = 'description'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['description_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


In [ ]:
# step3：过滤description_jieba，得到以空格分割的字符串

col_name = 'description_jieba'

def col_jieba_filter_fun(series):

    col_list_filter = []
    # 得到tag列表
    col_list = series[col_name]
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

jd_data['description_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)
jd_data

In [ ]:
# step4：将description_jieba_filter转成tfidf向量

def get_tfidf(df, col_name):
    # col_name必须是以空格分割的字符串
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    # 返回的是nparray
    vector = vectorizer.fit_transform(text)
    return pd.DataFrame(vector.toarray()), vectorizer

tfidf, vectorizer = get_tfidf(jd_data, 'description_jieba_filter')
tfidf

In [ ]:
vectorizer.vocabulary_

In [ ]:
# step5：将tfidf降维到n维

def get_tfidf_pca(tfidf, n=20):
    pca = PCA(n_components=n)
    tfidf_pca = pca.fit_transform(tfidf)
    tfidf_pca = pd.DataFrame(tfidf_pca)
    return tfidf_pca

tfidf_pca = get_tfidf_pca(tfidf, 10)
tfidf_pca

In [36]:
import pandas as pd

dict1 = {"流浪地球": [25,45,65,89],
        "唐人街探案":(89,56,89,17),
        "谋杀2":[19,46,49,79]}

index = ["第{}天".format(i) for i in range(1,5)]
df = pd.DataFrame(dict1, index)

df

,流浪地球,唐人街探案,谋杀2
第1天,25,89,19
第2天,45,56,46
第3天,65,89,49
第4天,89,17,79


In [47]:
def fun(series, col1, col2):
    return str(series[col1]) + str(series[col2])

df['new_col'] = df.apply(fun, axis=1, args=('唐人街探案', '谋杀2', ))
df

,流浪地球,唐人街探案,谋杀2,new_col
第1天,25,89,19,8919
第2天,45,56,46,5646
第3天,65,89,49,8949
第4天,89,17,79,1779
